In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder\
    .master("local[*]")\
        .appName('Test')\
            .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/02/19 20:45:59 WARN Utils: Your hostname, DESKTOP-SLEQT56 resolves to a loopback address: 127.0.1.1; using 172.25.13.138 instead (on interface eth0)
24/02/19 20:45:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/19 20:46:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


```
Leemos todos los archivos de una vez.
Como la estructura es anidada YEAR/MES hacemos */*/
```

In [3]:
df_green = spark.read.parquet('../data/raw/green/*/*/')
df_yellow = spark.read.parquet('../data/raw/yellow/*/*/')

In [9]:
mis_columns = {'green': df_green.columns, 'yellow': df_yellow.columns}
print(mis_columns)

{'green': ['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'], 'yellow': ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee']}


No todas las columnas de DataSet están compartidas, buscamos las comunes.

In [10]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

Unificamos algunas columnas

In [11]:
df_green = df_green.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime')\
    .withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime')
    
df_yellow = df_yellow.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')\
    .withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime')

Volvemos a Ejecutar la comparacion de columnas, ahora vemos Drop_off y pickup

In [12]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

Vemos que el orden no es el original, hacemos un artilugio

In [14]:
var_columns = df_yellow.columns
common_cols = []

for col in df_green.columns:
    if col in var_columns:
        common_cols.append(col)
        
print(common_cols)
        

['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'congestion_surcharge']


Ahora queremos seleccionar estas columnas de ambos DataFrames y agregamos una columna que indique a que DataSet pertenece cada registro

In [15]:
from pyspark.sql import functions as F

In [16]:
df_green_select = df_green.select(common_cols)\
    .withColumn('service_type', F.lit('green'))
    
df_yellow_select = df_yellow.select(common_cols)\
    .withColumn('service_type', F.lit('yellow'))

Unimos los dos DataFrames y calulamos los totales

In [17]:
df_trips_data = df_green_select.unionAll(df_yellow_select)

In [19]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



Ahora usamos SQL para ver los datos

In [20]:
df_trips_data.createOrReplaceTempView('trips_data')

In [52]:
spark.sql("""
          SELECT service_type, count(*) FROM trips_data 
          group by service_type
          """).show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [56]:
df_reporte = spark.sql("""
        SELECT 
        PULocationID AS reveneue_zone,
        service_type,
        
        -- Reveneue Calculation
        sum(fare_amount) as reveneue_monthly_fare,
        SUM(extra) as reveneue_monthly_extra
        
        from trips_data
        GROUP BY reveneue_zone, service_type
        order by reveneue_zone
                      """)

In [61]:
!mkdir '../data/reporte/'

In [68]:
df_reporte.coalesce(1).write.parquet("../data/reporte/mi_reporte", mode='overwrite')

In [69]:
mi_reporte = spark.read.parquet("../data/reporte/*")
mi_reporte.show()

+-------------+------------+---------------------+----------------------+
|reveneue_zone|service_type|reveneue_monthly_fare|reveneue_monthly_extra|
+-------------+------------+---------------------+----------------------+
|            1|       green|              3678.64|                   3.0|
|            1|      yellow|    400740.0900000001|    173.10000000000002|
|            2|       green|               212.66|                 14.25|
|            2|      yellow|              1753.83|                 23.75|
|            3|       green|   110828.77000000005|     5985.719999999999|
|            3|      yellow|   108866.59000000004|               4104.99|
|            4|       green|    20052.40999999998|               1134.55|
|            4|      yellow|   1049507.1299999913|     74716.56000000001|
|            5|       green|             14185.01|                137.25|
|            5|      yellow|             16786.27|                 70.75|
|            6|       green|          